In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import tqdm
from scipy.stats import levene

# import anova and tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests

In [2]:
def anova_function(features_df: pd.DataFrame, Metdata_column: str) -> pd.DataFrame:
    """
    This function will take in a dataframe and a metadata column and return the results of an anova and tukeyhsd test for each feature.


    Parameters
    ----------
    features_df : pd.DataFrame
        The dataframe containing the features with only one metadata column
    Metdata_column : str
        The name of the metadata column to be used for the anova test

    Returns
    -------
    pd.DataFrame
        A dataframe containing the results of the anova and tukeyhsd test for each feature
    """

    # anova and tukeyhsd for each feature
    # create a pandas data frame to store the results
    anova_results = pd.DataFrame()

    # loop through each feature
    for feature in tqdm.tqdm(features_df.columns[:-1]):
        # create a model
        model = ols(f"{feature} ~ C({Metdata_column})", data=features_df).fit()
        # create an anova table
        anova_table = sm.stats.anova_lm(model, typ=2)
        # create a tukeyhsd table
        tukeyhsd = pairwise_tukeyhsd(features_df[feature], features_df[Metdata_column])
        # get the f-statistic based p-value
        anova_p_value = anova_table["PR(>F)"][0]
        tmp = pd.DataFrame(
            tukeyhsd._results_table.data, columns=tukeyhsd._results_table.data[0]
        ).drop(0)
        tmp.reset_index(inplace=True, drop=True)
        # drop the first row
        tmp["feature"] = feature
        tmp["anova_p_value"] = anova_p_value
        tmp = pd.DataFrame(tmp)

        anova_results = pd.concat([anova_results, tmp], axis=0).reset_index(drop=True)
    return anova_results

In [3]:
file_path = pathlib.Path("../../data/5.converted_data/custom_aggregated_data.parquet")
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,Metadata_side,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,L,-0.178797,-0.178797,0.003558,-0.386796,0.280118,0.037998,0.434046,...,-0.627762,-0.644990,-0.903354,-0.890939,-0.896995,-0.902870,-0.896145,-0.898448,-0.898880,-0.896609
1,high,1,R,1.371763,1.371763,0.538195,0.288322,0.501618,4.163833,-1.361920,...,0.419551,0.406918,-0.188358,-0.159114,-0.189615,-0.186308,-0.218981,-0.225754,-0.219073,-0.225134
2,high,10,L,1.706234,1.706234,2.652373,3.280425,-1.992966,0.184630,-1.404376,...,0.841475,0.834574,1.149186,1.152200,1.078905,1.144848,1.040009,1.025740,1.045492,1.027617
3,high,10,R,0.771674,0.771674,-1.332747,0.164304,0.244371,0.173760,-1.885144,...,0.452480,0.436138,0.112178,0.131982,0.108602,0.125700,0.075717,0.071207,0.076477,0.067922
4,high,11,L,2.180858,2.180858,2.142686,1.838038,-0.077515,0.008650,0.876597,...,0.494813,0.490844,-0.016140,0.001041,-0.021997,-0.020992,-0.033934,-0.042635,-0.034983,-0.038763


In [4]:
# split the features and the metadata
metadata = df.columns.str.contains("Metadata")
# filter the metadata
metadata_df = df.loc[:, metadata]
# filter the features
features_df = df.loc[:, ~metadata]

## Anova for genotype only

In [5]:
anova_input_df = features_df.copy()
anova_input_df["Metadata_genotype"] = metadata_df["Metadata_genotype"]
anova_output_df = anova_function(anova_input_df, "Metadata_genotype")
print(anova_output_df.shape)
anova_output_df.head()

  0%|                                                                                                                                          | 0/244 [00:00<?, ?it/s]

  0%|▌                                                                                                                                 | 1/244 [00:00<00:28,  8.45it/s]

  1%|█                                                                                                                                 | 2/244 [00:00<00:27,  8.70it/s]

  1%|█▌                                                                                                                                | 3/244 [00:00<00:27,  8.82it/s]

  2%|██▏                                                                                                                               | 4/244 [00:00<00:27,  8.80it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 243/244 [00:27<00:00,  8.89it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:27<00:00,  8.87it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:27<00:00,  8.95it/s]

(732, 9)


,group1,group2,meandiff,p-adj,lower,upper,reject,feature,anova_p_value
0,high,unsel,-1.7236,0.0,-2.2266,-1.2205,True,AreaShape_Area,8.163020e-20
1,high,wt,-2.5954,0.0,-3.0938,-2.097,True,AreaShape_Area,8.163020e-20
2,unsel,wt,-0.8718,0.0003,-1.3749,-0.3688,True,AreaShape_Area,8.163020e-20
3,high,unsel,-1.5096,0.0,-1.8942,-1.125,True,AreaShape_CentralMoment_0_0,1.089545e-20
4,high,wt,-2.0158,0.0,-2.3969,-1.6347,True,AreaShape_CentralMoment_0_0,1.089545e-20


In [6]:
# save the results
output_file = pathlib.Path(
    "../../data/6.analysis_results/custom_aggregated_anova_results.parquet"
)
output_file.parent.mkdir(exist_ok=True, parents=True)
anova_output_df.to_parquet(output_file)

## Levene's test for homogeneity of variance across all groups

In [7]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
levene_test_results = {"feature": [], "levene_statistic": [], "levene_p_value": []}
for feature in tqdm.tqdm(features_df.columns):
    # calculate the levene test for each feature
    levene_results = levene(wt_df[feature], unsel_df[feature], high_df[feature])
    levene_test_results["feature"].append(feature)
    levene_test_results["levene_statistic"].append(levene_results.statistic)
    levene_test_results["levene_p_value"].append(levene_results.pvalue)

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df

  0%|                                                                                                                                          | 0/244 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:00<00:00, 3669.81it/s]

,feature,levene_statistic,levene_p_value
0,AreaShape_Area,2.664096,7.583990e-02
1,AreaShape_CentralMoment_0_0,4.808285,1.066760e-02
2,AreaShape_CentralMoment_0_1,5.060027,8.525926e-03
3,AreaShape_CentralMoment_0_2,9.782865,1.581992e-04
4,AreaShape_CentralMoment_0_3,6.034346,3.623435e-03
...,...,...,...
239,Texture_SumVariance_OP_3_03_256,7.546945,9.942484e-04
240,Texture_Variance_OP_3_00_256,22.209099,2.129331e-08
241,Texture_Variance_OP_3_01_256,8.906770,3.218403e-04
242,Texture_Variance_OP_3_02_256,22.013897,2.414564e-08


## Calculate the levenes test statistic for the equality of variances
## Pairwise

In [8]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
group_dict = {
    "high_vs_unsel": [high_df, unsel_df],
    "high_vs_wt": [high_df, wt_df],
    "unsel_vs_wt": [wt_df, unsel_df],
}


levene_test_results = {
    "feature": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
    "holm_bonferroni_p_value": [],
}

for feature in tqdm.tqdm(features_df.columns):
    levene_p_values = []
    for group_comparison in group_dict.keys():
        # calculate the levene test for each feature
        levene_results = levene(
            group_dict[group_comparison][0][feature],
            group_dict[group_comparison][1][feature],
        )
        levene_test_results["feature"].append(feature)
        levene_test_results["levene_statistic"].append(levene_results.statistic)
        levene_test_results["levene_p_value"].append(levene_results.pvalue)
        levene_test_results["group"].append(group_comparison)
        levene_p_values.append(levene_results.pvalue)
    levene_holm_bonferroni_p_values = multipletests(levene_p_values, method="holm")[1]
    # run holm-bonferroni correction on the p-values for each feature
    [
        levene_test_results["holm_bonferroni_p_value"].append(p_value)
        for p_value in levene_holm_bonferroni_p_values
    ]

  0%|                                                                                                                                          | 0/244 [00:00<?, ?it/s]

  1%|█▌                                                                                                                                | 3/244 [00:00<00:09, 26.72it/s]

  3%|███▋                                                                                                                              | 7/244 [00:00<00:08, 29.15it/s]

  5%|█████▊                                                                                                                           | 11/244 [00:00<00:07, 29.92it/s]

  6%|███████▉                                                                                                                         | 15/244 [00:00<00:07, 30.44it/s]

...

 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 232/244 [00:07<00:00, 28.74it/s]

 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 235/244 [00:07<00:00, 28.71it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 239/244 [00:07<00:00, 29.45it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 243/244 [00:08<00:00, 29.92it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:08<00:00, 29.99it/s]

In [9]:
levene_test_results_df = pd.DataFrame(levene_test_results)

# sort the levene test results levene_test_results_df
# change the levene p-value to a float
levene_test_results_df["levene_p_value"] = levene_test_results_df[
    "levene_p_value"
].astype(float)
levene_test_results_df = levene_test_results_df.sort_values(
    ["feature", "group"], ascending=[True, True]
)
levene_test_results_df

,feature,levene_statistic,levene_p_value,group,holm_bonferroni_p_value
0,AreaShape_Area,1.312482,2.570925e-01,high_vs_unsel,0.427459
1,AreaShape_Area,1.583091,2.137293e-01,high_vs_wt,0.427459
2,AreaShape_Area,5.121513,2.775241e-02,unsel_vs_wt,0.083257
3,AreaShape_CentralMoment_0_0,0.757577,3.880131e-01,high_vs_unsel,0.388013
4,AreaShape_CentralMoment_0_0,13.490942,5.513078e-04,high_vs_wt,0.001654
...,...,...,...,...,...
727,Texture_Variance_OP_3_02_256,31.005530,8.311402e-07,high_vs_wt,0.000002
728,Texture_Variance_OP_3_02_256,20.806708,3.042424e-05,unsel_vs_wt,0.000061
729,Texture_Variance_OP_3_03_256,1.249779,2.686365e-01,high_vs_unsel,0.268636
730,Texture_Variance_OP_3_03_256,6.845386,1.150109e-02,high_vs_wt,0.023002


In [10]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "custom_aggregated_levene_test_results.csv"
)
levene_test_results_df.to_csv(levene_test_results_path)

### Calculate the levene test statistic for the aggregated data across feature types and genotypes

In [11]:
data_path = pathlib.Path(
    "../../data/5.converted_data/custom_aggregated_data.parquet"
).resolve(strict=True)
# Read the data
data = pd.read_parquet(data_path)

# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]


# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df.head()
# Separate the feature into different parts
features_long_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = features_long_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
features_long_df["Metadata_genotype"] = features_long_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
features_long_df.head()

,Metadata_genotype,feature,value,feature_group,measurement,bone,parameter1,parameter2,parameter3
0,High-Severity,AreaShape_Area,-0.178797,AreaShape,Area,None,None,None,None
1,High-Severity,AreaShape_Area,1.371763,AreaShape,Area,None,None,None,None
2,High-Severity,AreaShape_Area,1.706234,AreaShape,Area,None,None,None,None
3,High-Severity,AreaShape_Area,0.771674,AreaShape,Area,None,None,None,None
4,High-Severity,AreaShape_Area,2.180858,AreaShape,Area,None,None,None,None


In [12]:
# break each genotype and featuretype into a separate dataframe
high_df = features_long_df[features_long_df["Metadata_genotype"] == "High-Severity"]
unsel_df = features_long_df[features_long_df["Metadata_genotype"] == "Mid-Severity"]
wt_df = features_long_df[features_long_df["Metadata_genotype"] == "Wild Type"]

# each feature group
high_df_AreaShape = high_df[high_df["feature_group"] == "AreaShape"]
high_df_Intensity = high_df[high_df["feature_group"] == "Intensity"]
high_df_Neighbors = high_df[high_df["feature_group"] == "Neighbors"]
high_df_radial = high_df[high_df["feature_group"] == "RadialDistribution"]
high_df_Granularity = high_df[high_df["feature_group"] == "Granularity"]

unsel_df_AreaShape = unsel_df[unsel_df["feature_group"] == "AreaShape"]
unsel_df_Intensity = unsel_df[unsel_df["feature_group"] == "Intensity"]
unsel_df_Neighbors = unsel_df[unsel_df["feature_group"] == "Neighbors"]
unsel_df_radial = unsel_df[unsel_df["feature_group"] == "RadialDistribution"]
unsel_df_Granularity = unsel_df[unsel_df["feature_group"] == "Granularity"]

wt_df_AreaShape = wt_df[wt_df["feature_group"] == "AreaShape"]
wt_df_Intensity = wt_df[wt_df["feature_group"] == "Intensity"]
wt_df_Neighbors = wt_df[wt_df["feature_group"] == "Neighbors"]
wt_df_radial = wt_df[wt_df["feature_group"] == "RadialDistribution"]
wt_df_Granularity = wt_df[wt_df["feature_group"] == "Granularity"]

# levene test for each feature group
levene_test_results = {
    "feature_group": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}

group_dict = {
    "AreaShape": {
        "high_area_v_unsel_area": [high_df_AreaShape, unsel_df_AreaShape],
        "high_area_v_wt_area": [high_df_AreaShape, wt_df_AreaShape],
        "unsel_area_v_wt_area": [wt_df_AreaShape, unsel_df_AreaShape],
    },
    "Intensity": {
        "high_intensity_v_unsel_intensity": [high_df_Intensity, unsel_df_Intensity],
        "high_intensity_v_wt_intensity": [high_df_Intensity, wt_df_Intensity],
        "unsel_intensity_v_wt_intensity": [wt_df_Intensity, unsel_df_Intensity],
    },
    "Neighbors": {
        "high_neighbors_v_unsel_neighbors": [high_df_Neighbors, unsel_df_Neighbors],
        "high_neighbors_v_wt_neighbors": [high_df_Neighbors, wt_df_Neighbors],
        "unsel_neighbors_v_wt_neighbors": [wt_df_Neighbors, unsel_df_Neighbors],
    },
    "RadialDistribution": {
        "high_radial_v_unsel_radial": [high_df_radial, unsel_df_radial],
        "high_radial_v_wt_radial": [high_df_radial, wt_df_radial],
        "unsel_radial_v_wt_radial": [wt_df_radial, unsel_df_radial],
    },
    "Granularity": {
        "high_granularity_v_unsel_granularity": [
            high_df_Granularity,
            unsel_df_Granularity,
        ],
        "high_granularity_v_wt_granularity": [high_df_Granularity, wt_df_Granularity],
        "unsel_granularity_v_wt_granularity": [wt_df_Granularity, unsel_df_Granularity],
    },
}

for feature_group in tqdm.tqdm(group_dict.keys()):
    for group in group_dict[feature_group].keys():
        if not group == "all":
            levene_results = levene(
                group_dict[feature_group][group][0]["value"],
                group_dict[feature_group][group][1]["value"],
            )
            # calculate the variance for each feature group

            levene_test_results["feature_group"].append(feature_group)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            pass

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df.head()

  0%|                                                                                                                                            | 0/5 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1299.35it/s]

,feature_group,levene_statistic,levene_p_value,group
0,AreaShape,47.175905,7.222090e-12,high_area_v_unsel_area
1,AreaShape,113.242704,3.432556e-26,high_area_v_wt_area
2,AreaShape,7.530054,6.087899e-03,unsel_area_v_wt_area
3,Intensity,0.931100,3.348608e-01,high_intensity_v_unsel_intensity
4,Intensity,59.572829,3.351570e-14,high_intensity_v_wt_intensity


In [13]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "custom_aggregated_levene_test_results_feature_types.csv"
)
levene_test_results_df.to_csv(levene_test_results_path, index=False)

In [14]:
# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]

# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
# get the variance for each feature for each genotype
features_long_df
all_features_var = features_long_df.groupby(["Metadata_genotype", "feature"]).var()
# reset the index
all_features_var = all_features_var.reset_index()
all_features_var.rename(columns={"value": "variance"}, inplace=True)

# save the variance results
var_each_feature_path = pathlib.Path(
    out_dir / "custom_aggregated_variance_results_each_feature.csv"
)
all_features_var.to_csv(var_each_feature_path, index=False)
all_features_var.head()

,Metadata_genotype,feature,variance
0,high,AreaShape_Area,0.516762
1,high,AreaShape_CentralMoment_0_0,0.516762
2,high,AreaShape_CentralMoment_0_1,2.361025
3,high,AreaShape_CentralMoment_0_2,1.147158
4,high,AreaShape_CentralMoment_0_3,0.716030


In [15]:
# get the variance for each feature group
var_df = features_long_df.groupby(["Metadata_genotype", "feature"]).var().reset_index()
var_df.head()
# change the value column name to variance
var_df.rename(columns={"value": "variance"}, inplace=True)

In [16]:
var_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = var_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
var_df["Metadata_genotype"] = var_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
var_df
var_df = var_df.drop(
    columns=["feature", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
)
var_df
# save the variance results
var_path = pathlib.Path(
    out_dir / "custom_aggregated_variance_results_feature_types.csv"
)
var_df.to_csv(var_path, index=False)

In [17]:
# get the mean and stdev for each feature group's variance
var_df = (
    var_df.groupby(["Metadata_genotype", "feature_group"])
    .agg(["mean", "std", "max", "min", "count"])
    .reset_index()
)
# ungroup the columns
var_df.columns = ["_".join(col).strip() for col in var_df.columns.values]
# rename the Metadata_genotype_ column and the feature_group_ column
var_df.rename(
    columns={
        "Metadata_genotype_": "Metadata_genotype",
        "feature_group_": "feature_group",
    },
    inplace=True,
)
var_df

,Metadata_genotype,feature_group,variance_mean,variance_std,variance_max,variance_min,variance_count
0,High-Severity,AreaShape,0.774588,0.669970,3.855982,0.000005,98
1,High-Severity,Granularity,0.278190,0.357954,0.981199,0.015287,6
2,High-Severity,Intensity,0.529629,0.281565,1.157014,0.129043,15
3,High-Severity,Neighbors,0.730238,0.237898,0.953424,0.479949,3
4,High-Severity,RadialDistribution,0.670404,0.430295,1.751219,0.000000,70
5,High-Severity,Texture,0.300315,0.206077,0.719217,0.032302,52
6,Mid-Severity,AreaShape,1.112337,0.783353,3.934014,0.165658,98
7,Mid-Severity,Granularity,0.792799,0.681926,1.982012,0.191056,6
8,Mid-Severity,Intensity,0.699825,0.633322,2.091309,0.076053,15
9,Mid-Severity,Neighbors,1.909325,1.552501,3.634385,0.624438,3


In [18]:
# save the variance results
var_path = pathlib.Path(
    out_dir / "custom_aggregated_variance_results_feature_types_stats.csv"
)
var_df.to_csv(var_path, index=False)